# REINFORCE for discrete reward

Данный ноутбук содержит эксперимент,
включающий в себя использовать дискретную награду
    и адаптировать метод [REINFORCE](https://arxiv.org/abs/2402.14740)
для такой награды в задаче alignment.

RLHF (Reinforcement Learning though Human Feedback) --- метод,
предназначен для выравнивания ответов языковой модели с потребностями прользователя.
Этот процесс включает в себя следующие этапы:
1. SFT (Supervised Fine Tuning) --- обучение с учителем на датасете в формате чата.
2. RM (Reward Model) --- модель, предсказывающая награду, которую модель получит при
   конкретном запросе и ответе.
3. RL (Reinforcement Learning) --- обучение с подкреплением, где в качестве награды используется RM.
   Часто в качестве алгоритма выступает [PPO](https://arxiv.org/abs/1707.06347). Однако данный подход
   слишком тяжеловесный, поэтому в этом ноутбуке рассматривается метод REINFORCE и его адаптация.

__Примечание:__ в этом ноутбуке представлено большое количество строк кода. Однако, для понимания происходящего,
достаточно лишь читать содержимое markdown ячеек, а коду обращаться только за уточнениями и дополнительными подробностями.

## Предварительная подготовка

### Обьявим конфиг

Здесь можно изменить batch size, максимальную длину последовательности и другие параметры обучения.

In [1]:
cfg = {
    "attention_mechanism": "flash_attention_2",
    "cache_dir": "data/",
    "dataset": "juyoungml/HelpSteer2-binarized",
    "DRM_batch_size": 8,
    "DRM_epochs": 1,
    "DRM_lr": 5e-5,
    "DRM_model": "data/discrete_reward_model",
    "DRM_n_classes": 10,
    "DRM_train": False,
    "DRL_batch_size": 12,
    "DRL_epochs": 1,
    "DRL_lr": 5e-5,
    "DRL_model": "data/discrete_rl_model",
    "DRL_reward_optimism": 0.3,
    "DRL_rollout_batch_size": 1,
    "DRL_train": False,
    "DRL_warmup_ratio": 0.03,
    "generation_config": {
        "do_sample": True,
        "temperature": 1.0,
        "top_k": 100,
        "max_new_tokens": 512,
    },
    "kl_coef": 0.1,
    "logger": "wandb",
    "max_token_seq_length": 2048,
    "RM_batch_size": 8,
    "RM_epochs": 1,
    "RM_lr": 5e-5,
    "RM_model": "data/reward_model",
    "RM_train": False,
    "RL_batch_size": 12,
    "RL_epochs": 1,
    "RL_lr": 5e-5,
    "RL_model": "data/rl_model",
    "RL_rollout_batch_size": 1,
    "RL_train": False,
    "RL_warmup_ratio": 0.03,
    "SFT_model": "HuggingFaceTB/SmolLM2-135M-Instruct",
    "trainer_dir": "data/trainer_output",
    "wandb_project": "discrete_reinforce",
}

### Необходимые библиотеки

In [2]:
import inspect
import os
from pathlib import Path
from typing import Any, Optional

import datasets
import numpy as np
import torch
from tqdm.auto import tqdm, trange
import trl
import transformers
import wandb

In [3]:
os.environ["WANDB_PROJECT"] = cfg["wandb_project"]

### Загружаем датасет

В постановке задачи был предложен набор данных `esfrankel17/HelpSteer2_binarized`,
однако именно такой набор не существует на Hugging Face,
поэтому был выбран набор [`juyoungml/HelpSteer2-binarized`](https://huggingface.co/datasets/juyoungml/HelpSteer2-binarized),
на который указывает ссылка из задания.

Этот набор уже разбит на тренировочную и валидационную части, размеры которых $7224$ и $373$ соответственно.

In [4]:
data = datasets.load_dataset(cfg["dataset"], cache_dir=cfg["cache_dir"])
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'chosen_score', 'rejected_score', 'chosen_rationale', 'rejected_rationale', 'score_diff', 'difficulty'],
        num_rows: 7224
    })
    validation: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'chosen_score', 'rejected_score', 'chosen_rationale', 'rejected_rationale', 'score_diff', 'difficulty'],
        num_rows: 373
    })
})

### Загружаем токенайзер

Токенайзер используется от выбранной SFT модели
[`HuggingFaceTB/SmolLM2-135M-Instruct`](https://huggingface.co/HuggingFaceTB/SmolLM2-135M-Instruct).

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(cfg["SFT_model"], cache_dir=cfg["cache_dir"])
tokenizer.padding_side = "left"

### Преобразуем данные в формат чата

SFT модель требует данные в специфичном формате чата.
К нему и приводятся запросы и ответы.

In [6]:
def make_chat_from_prompt(prompt):
    messages = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        continue_final_message=True,
    )

def make_chat_from_conversation(prompt, response):
    messages = [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response}
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        continue_final_message=False,
    )


def tokenize_chat(batch):
    batch["chosen"] = [make_chat_from_conversation(prompt, response) for prompt, response in zip(batch["prompt"], batch["chosen"])]
    batch["rejected"] = [make_chat_from_conversation(prompt, response) for prompt, response in zip(batch["prompt"], batch["rejected"])]
    batch["prompt"] = [make_chat_from_prompt(prompt) for prompt in batch["prompt"]]

    tokens = tokenizer(batch["prompt"], truncation=True, max_length=cfg["max_token_seq_length"])
    batch["input_ids"] = tokens["input_ids"]
    batch["attention_mask"] = tokens["attention_mask"]
    
    return batch

In [7]:
data = data.map(tokenize_chat, batched=True)

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

## Continuous Reward Model

RM представляет из себя основу от SFT с головой, выдающей один скаляр.
Функция потерь выглядит следующим образом:
$$
\mathcal{L}_{RM} = -\log \sigma (r_{\psi}(x, y_w) - r_{\psi}(x, y_l))
$$
где $x$~--- входной промпт, $y_w$~--- предпочитаемый ответ, $y_l$~--- плохой
ответ, а $r_{\psi}$~--- модель награды.

__Примечание:__ в статье про RLOO в определении функции потерь вероятно
допущена опечатка, а именно внутри сигмоиды наодится ещё один логарифм.
Для подтверждения этого предположения ссылаюсь 
на реализацию [RewardTrainer](https://github.com/huggingface/trl/blob/main/trl/trainer/reward_trainer.py)
и на [Training language models to follow instructions
with human feedback](https://arxiv.org/abs/2203.02155).

### Инициализация Reward Model

Требуемая модель загружается с диска, если она уже обучена, или инициализируется
весами SFT в противном случае.

In [8]:
if cfg["RM_train"]:
    rm_model = transformers.AutoModelForSequenceClassification.from_pretrained(
        cfg["SFT_model"],
        num_labels=1,
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )
else:
    rm_model = transformers.AutoModelForSequenceClassification.from_pretrained(
        cfg["RM_model"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


### Обучени Reward Model

Обучение происходит только в случае необходимости.

In [9]:
rm_cfg = trl.RewardConfig(
    output_dir=cfg["trainer_dir"],
    run_name="reward_continuous",
    report_to=cfg["logger"],
    eval_strategy="epoch",
    max_length=cfg["max_token_seq_length"],
    per_device_train_batch_size=cfg["RM_batch_size"],
    per_device_eval_batch_size=cfg["RM_batch_size"],
    learning_rate=cfg["RM_lr"],
    num_train_epochs=cfg["RM_epochs"],
    bf16=True
)

rm_trainer = trl.RewardTrainer(
    model=rm_model,
    args=rm_cfg,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    processing_class=tokenizer,
)

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7224 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

Filter:   0%|          | 0/373 [00:00<?, ? examples/s]

In [10]:
if cfg["RM_train"]:
    rm_trainer.train()
    rm_model.save_pretrained(cfg["RM_model"])
    wandb.finish()

del rm_trainer

## Continuous REINFORCE

Алгоритм REINFORCE оптимизируется подъёмом по градиенту
$$
\mathbb{E}_{x \sim \mathcal{D},\, y \sim \pi_{\theta}(. | x)} [(R(x,y) - b) \nabla_{\theta} \log{\pi_{\theta}(y | x)}],
$$
$$
R(x, y) = r_{\psi}(x, y) - \beta \log\frac{\pi_{\theta}(y|x)}{\pi_{\text{ref}}(y,x)},
$$
где $\pi_{\theta}$, $\pi_{\text{ref}}$ --- обучаемая политика и политика, основанная на SFT модели
соответственно, $\beta$ --- гиперпараметр, показвывающий допустимое отклонение обучаемой политики от исходной.
В приведенной формуле тажже использовался бейзлайн $b = \frac{1}{R} \sum_{i=1}^{S} R(x^i, y^i)$ --- усреднение
всех наград за всё время обучения. Он необходим для снижения дисперсии градиента.

### Реализуем REINFORCETrainer

Класс [RLOOTrainer](https://github.com/huggingface/trl/blob/main/trl/trainer/rloo_trainer.py) из 
библиотеки [trl](https://github.com/huggingface/trl) не подходит, так как 
эта реализация отличается от описанной в [статье](https://arxiv.org/abs/2402.14740):
* Библиотечная реализация использует клипинг для обновления политики,
что является избыточным, а его отсутствие не приводит к ухудшению качества.
* Преимущество считается отдельно для каждого токена, если не указать
параметр `reward_level_kl=False`.
* Веса обновляются несколько раз на преимуществах.

__Примечание:__ в этой реализация REINFORCE также поддерживает дискретные награды
(подробности смотри с соответствующем разделе ноутбука).

In [11]:
class ReinforceTrainer(transformers.Trainer):
    def __init__(
        self,
        *args,
        generation_config: dict[str, Any],
        ref_model: torch.nn.Module,
        reward_model: torch.nn.Module,
        kl_coef: float,
        rollout_batch_size: int,
        reward_optimism: float = 0.5,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.model.generation_config.update(**generation_config)
        self.ref_model = ref_model.to(self.model.device)
        self.ref_model.eval()
        self.reward_model = reward_model.to(self.model.device)
        self.reward_model.eval()

        self.kl_coef = kl_coef
        self.reward_optimism = reward_optimism
        self.rollout_batch_size = rollout_batch_size
        self.moving_average_reward = 0.0
        self.training_steps_counter = 0

    def _get_reward_quantile(self, rewards: torch.Tensor) -> torch.Tensor:
        np_rewards = rewards.cpu().float().numpy()
        cdf = np.cumsum(np_rewards, axis=1)
        quantiles = np.zeros(cdf.shape[0])
        for i in range(cdf.shape[0]):
            quantiles[i] = np.interp(
                self.reward_optimism,
                cdf[i],
                np.arange(cdf.shape[1]),
                left=0,
                right=cdf.shape[1] - 1
            )
        return torch.tensor(quantiles, device=rewards.device, dtype=rewards.dtype)
        
    def _get_reward(self, query_responses: torch.Tensor) -> torch.Tensor:
        attention_mask = query_responses != self.processing_class.pad_token_id
        rewards = self.reward_model(
            input_ids=query_responses,
            attention_mask=attention_mask
        ).logits
        if rewards.ndim == 2 and rewards.shape[1] > 1:
            rewards = torch.nn.functional.softmax(rewards, dim=1)
            rewards = self._get_reward_quantile(rewards)
            
        return rewards

    def _calculate_log_probs(
        self,
        model: torch.nn.Module,
        query_responses: torch.Tensor,
        query_len: int
    ) -> torch.Tensor:
        attention_mask = query_responses != model.generation_config.pad_token_id
        logits = model(query_responses, attention_mask=attention_mask).logits
        logprobs = trl.trainer.utils.selective_log_softmax(
            logits[:, :-1], query_responses[:, 1:]
        )
        return logprobs[:, query_len - 1:].sum(1)

    def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
        queries = inputs["input_ids"]
        attention_masks = inputs["attention_mask"]
        queries_len = queries.shape[1]
        
        loss = 0.0
        all_rewards = []
        all_kl_divergences = []
        for _ in range(self.rollout_batch_size):
            with torch.no_grad():
                query_responses = self.model.generate(
                    input_ids=queries,
                    attention_mask=attention_masks
                )
        
            logprobs = self._calculate_log_probs(
                self.model, query_responses, queries_len
            )
            with torch.no_grad():
                ref_logprobs = self._calculate_log_probs(
                    self.ref_model, query_responses, queries_len
                )

                kl_divergence = logprobs.detach() - ref_logprobs
                rewards = self._get_reward(query_responses)
                rewards_normalized = rewards - self.kl_coef * kl_divergence

            all_rewards.append(rewards)
            all_kl_divergences.append(kl_divergence)
        
            self.training_steps_counter += 1
            self.moving_average_reward += rewards_normalized.mean().item()
            baseline = self.moving_average_reward / self.training_steps_counter
            advantages = rewards_normalized - baseline

            loss = loss - torch.mean(logprobs * advantages)

        loss /= self.rollout_batch_size
        self.log({
            "loss": loss.item(),
            "reward": torch.stack(all_rewards).mean().item(),
            "kl_divergence": torch.stack(all_kl_divergences).mean().item(),
            "moving_average_reward": self.moving_average_reward / self.training_steps_counter,
        })

        torch.cuda.empty_cache()
        return loss

    def prediction_step(
        self,
        model: torch.nn.Module,
        inputs: dict[str, torch.Tensor],
        prediction_loss_only: bool,
        ignore_keys: Optional[list[str]] = None,
    ) -> tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:
        queries = inputs["input_ids"]
        attention_masks = inputs["attention_mask"]
        queries_len = queries.shape[1]
        with torch.no_grad():
            query_responses = self.model.generate(
                input_ids=queries,
                attention_mask=attention_masks
            )
            logprobs = self._calculate_log_probs(
                self.model, query_responses, queries_len
            )
            ref_logprobs = self._calculate_log_probs(
                self.ref_model, query_responses, queries_len
            )

            kl_divergence = logprobs.detach() - ref_logprobs
            rewards = self._get_reward(query_responses)
            rewards_normalized = rewards - self.kl_coef * kl_divergence
        
            baseline = self.moving_average_reward / self.training_steps_counter
            advantages = rewards_normalized - baseline

            loss = -torch.mean(logprobs * advantages)

        torch.cuda.empty_cache()
        return loss, None, None

### Инициализация модели

In [12]:
rl_ref_model = transformers.AutoModelForCausalLM.from_pretrained(
    cfg["SFT_model"],
    attn_implementation=cfg["attention_mechanism"],
    torch_dtype=torch.bfloat16,
)
if cfg["RL_train"]:
    rl_model = transformers.AutoModelForCausalLM.from_pretrained(
        cfg["SFT_model"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )
else:
    rl_model = transformers.AutoModelForCausalLM.from_pretrained(
        cfg["RL_model"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )

### Обучение модели

Будем использовать SGD в качестве оптимизатора, чтобы сократить объём требуемой памяти.
Кроме того, исходя из приведённых в статье формул, в ней использовался именно этот
оптимизатор.

In [13]:
rl_config = transformers.TrainingArguments(
    output_dir=cfg["trainer_dir"],
    run_name="rl_continuous",
    report_to=cfg["logger"],
    learning_rate=cfg["RL_lr"],
    warmup_ratio=cfg["RL_warmup_ratio"],
    per_device_train_batch_size=cfg["RL_batch_size"],
    per_device_eval_batch_size=cfg["RL_batch_size"],
    num_train_epochs=cfg["RL_epochs"],
    eval_strategy="epoch",
    bf16=True,
)

rl_trainer = ReinforceTrainer(
    model=rl_model,
    optimizer_cls_and_kwargs=(torch.optim.SGD, {"lr": cfg["RL_lr"]}),
    args=rl_config,
    ref_model=rl_ref_model,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    processing_class=tokenizer,
    reward_model=rm_model,
    generation_config=cfg["generation_config"],
    kl_coef=cfg["kl_coef"],
    rollout_batch_size=cfg["RL_rollout_batch_size"],
)

In [14]:
if cfg["RL_train"]:
    rl_trainer.train()
    rl_model.save_pretrained(cfg["RL_model"])
    wandb.finish()

del rl_trainer

### Train Reward
![train_reward](imgs/rl_reward.png)

### Moving average baseline

![rl_baseline](imgs/rl_baseline.png)

Можно заметить, что бейзлайн в виде скользящего среднего работает хорошо,
постепенно выходя на оптимальную величину.

С другой стороны, модель не смогла хорошо обучиться. Это можно объяснить
`rollout_batch_size=1`, что приводит к тому, что модель не успевает подстроиться
под промпты.

## Discrete Reward Model (DRM)

Пусть теперь модель награды возвращает не число, а вероятности классов $r_{\psi}(y = k)$.
Выведем подходящую функцию потерь:
$$
\mathcal{L} = -p(y_w > y_l) = -\sum_{i=1}^K p(y_w = i)p(y_l < i) = -\sum_{i=1}^K \sum_{j=1}^{i-1} p(y_w = i)p(y_l = j),
$$
где $K$ --- количнство классов наград. Для улучшения сходимости (потенциальной), будем
оптимизировать $\log{\mathcal{L}}$.

### Реализация DRM

Для простоты скопируем и слегка модифицируем методы из `RewardTrainer`.

In [15]:
class DiscreteRewardTrainer(trl.RewardTrainer):
    def compute_loss(
        self,
        model: torch.nn.Module,
        inputs: dict[str, torch.Tensor | Any],
        return_outputs=False,
        num_items_in_batch=None,
    ) -> torch.Tensor | tuple[torch.Tensor, dict[str, torch.Tensor]]:
        probs_chosen = torch.nn.functional.softmax(model(
            input_ids=inputs["input_ids_chosen"],
            attention_mask=inputs["attention_mask_chosen"],
            return_dict=True,
        )["logits"], 1)
        probs_rejected = torch.nn.functional.softmax(model(
            input_ids=inputs["input_ids_rejected"],
            attention_mask=inputs["attention_mask_rejected"],
            return_dict=True,
        )["logits"], 1)
        
        cum_density_rejected = torch.cumsum(probs_rejected, 1) - probs_rejected[:, :1]
        loss = -(probs_chosen * cum_density_rejected).sum(1).log().mean()
        
        if return_outputs:
            return loss, {
                "probs_chosen": probs_chosen,
                "probs_rejected": probs_rejected,
            }
        return loss

    def prediction_step(
        self,
        model: torch.nn.Module,
        inputs: dict[str, torch.Tensor | Any],
        prediction_loss_only: bool,
        ignore_keys: Optional[list[str]] = None,
    ) -> tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:
        inputs = self._prepare_inputs(inputs)
        if ignore_keys is None:
            if hasattr(self.model, "config"):
                ignore_keys = getattr(self.model.config, "keys_to_ignore_at_inference", [])
            else:
                ignore_keys = []

        with torch.no_grad():
            loss, probs_dict = self.compute_loss(model, inputs, return_outputs=True)

        if prediction_loss_only:
            return (loss, None, None)

        cum_density_rejected = torch.cumsum(probs_dict["probs_rejected"], 1) - probs_dict["probs_rejected"][:, :1]
        select_probs = (probs_dict["probs_chosen"] * cum_density_rejected).sum(1)
        probs = torch.zeros([select_probs.shape[0], 2])
        probs[:, 0] = select_probs
        probs[:, 1] = 1 - select_probs

        labels = torch.zeros(probs.shape[0])
        labels = self._prepare_inputs(labels)
        
        return loss, probs, labels

### Инициализация модеоли

In [16]:
if cfg["DRM_train"]:
    drm_model = transformers.AutoModelForSequenceClassification.from_pretrained(
        cfg["SFT_model"],
        num_labels=cfg["DRM_n_classes"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )
else:
    drm_model = transformers.AutoModelForSequenceClassification.from_pretrained(
        cfg["DRM_model"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )

### Обучение модели

In [17]:
drm_cfg = trl.RewardConfig(
    output_dir=cfg["trainer_dir"],
    run_name="reward_discrete",
    report_to=cfg["logger"],
    eval_strategy="epoch",
    max_length=cfg["max_token_seq_length"],
    per_device_train_batch_size=cfg["DRM_batch_size"],
    per_device_eval_batch_size=cfg["DRM_batch_size"],
    learning_rate=cfg["DRM_lr"],
    num_train_epochs=cfg["DRM_epochs"],
    bf16=True
)

drm_trainer = DiscreteRewardTrainer(
    model=drm_model,
    args=drm_cfg,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    processing_class=tokenizer,
)

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7224 [00:00<?, ? examples/s]

In [18]:
if cfg["DRM_train"]:
    drm_trainer.train()
    drm_model.save_pretrained(cfg["DRM_model"])
    wandb.finish()

del drm_trainer

## REINFORCE с вероятностями дискретных наград

Самый простой и очевидный способ интерграции распределения награды в REINFORCE --- вычисление какой-нибудь статистики из распределения
и использование её в качестве награды (например среднее или квантиль). Кватнили
особенно интересны, так как позволяют регулировать консервативность политики.
Например, требуется оптимизировать не награду, которую модель получает в среднем.
а некоторую нижнюю границу.
$$
R(x, y) = S(r_{\psi}(\cdot | x, y)) - \log{\frac{\pi_{\theta}(y|x)}{\pi_{\text{ref}}(y|x)}},
$$
где $S$ --- квантиль распределения.

### Инициализация модели

In [19]:
if cfg["DRL_train"]:
    drl_model = transformers.AutoModelForCausalLM.from_pretrained(
        cfg["SFT_model"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )
else:
    drl_model = transformers.AutoModelForCausalLM.from_pretrained(
        cfg["DRL_model"],
        attn_implementation=cfg["attention_mechanism"],
        torch_dtype=torch.bfloat16,
    )

### Обучение модели

In [20]:
drl_config = transformers.TrainingArguments(
    output_dir=cfg["trainer_dir"],
    run_name="rl_discrete",
    report_to=cfg["logger"],
    learning_rate=cfg["DRL_lr"],
    warmup_ratio=cfg["DRL_warmup_ratio"],
    per_device_train_batch_size=cfg["DRL_batch_size"],
    per_device_eval_batch_size=cfg["DRL_batch_size"],
    num_train_epochs=cfg["DRL_epochs"],
    eval_strategy="epoch",
    bf16=True,
)

drl_trainer = ReinforceTrainer(
    model=drl_model,
    optimizer_cls_and_kwargs=(torch.optim.SGD, {"lr": cfg["DRL_lr"]}),
    args=rl_config,
    ref_model=rl_ref_model,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    processing_class=tokenizer,
    reward_model=drm_model,
    generation_config=cfg["generation_config"],
    kl_coef=cfg["kl_coef"],
    rollout_batch_size=cfg["DRL_rollout_batch_size"],
    reward_optimism=cfg["DRL_reward_optimism"],
)

In [21]:
if cfg["DRL_train"]:
    drl_trainer.train()
    drl_model.save_pretrained(cfg["DRL_model"])
    wandb.finish()

del drl_trainer

### Train Reward
![drl_train_reward](imgs/drl_reward.png)

Аналогичная предущему случаю ситуация.

## Сравнение моделей

In [22]:
@torch.no_grad
def eval_rlhf_model(model, reward_model, dataset):
    rewards = []
    for i in trange(0, len(dataset), cfg["RL_batch_size"]):
        batch = dataset[i : i + cfg["RL_batch_size"]]
        tokens = tokenizer.pad(batch)
        
        output = model.generate(
            input_ids=torch.tensor(tokens["input_ids"]).to(model.device),
            attention_mask=torch.tensor(tokens["attention_mask"]).to(model.device),
        )
        reward = reward_model(output).logits
        rewards.append(reward)
        
        del output
        torch.cuda.empty_cache()
    return rewards

In [23]:
reward_sft = eval_rlhf_model(rl_ref_model, rm_model, data["validation"])

  0%|          | 0/32 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
